# Spatial Distribution Exercise

The goal of this tutorial is learn how to characterize what cells exist in the visual cortex volume and how they are distributed across space and cell types.

## Note if running on Google Colab

If you are running in Google colab, you will need to install a couple of packages to make it work. Just run `pip install caveclient standard_transform` in a cell at the begining of the notebook.

Also, you will have to get your token from the [CAVE website](https://global.daf-apis.com/sticky_auth/settings/tokens) and set it by passing an auth token to the CAVEclient directly via `client = CAVEclient(datastack, auth_token=auth_token)`.

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import spatial

from caveclient import CAVEclient

datastack = 'minnie65_public'
client = CAVEclient(datastack)

## Getting an overview of the cell types in the data

Download `aibs_metamodel_mtypes_v661_v2`, a table of cell type predictions throughout the dataset.
Look at the effect of the `split_positions` parameter, which can be set to True or False.

For simplicity, we will simplify the excitatory cell types in this prediction table using the pandas `replace` function, which takes a dictionary and a dataframe column and maps keys to values.
Assuming our cell type dataframe is called `soma_df`, we want to do:

```python
ct_simplify = {
    "L2a": "L2", # Layer 2 excitatory cell
    "L2b": "L2",
    "L2c": "L3", # Layer 3 excitatory cell
    "L3a": "L3",
    "L3b": "L3",
    "L4a": "L4", # Layer 4 excitatory cell
    "L4b": "L4",
    "L4c": "L4",
    "L5a": "L5it", # Layer 5 cortico-cortical (IT) cell
    "L5b": "L5it",
    "L5ET": "L5et", # Layer 5 subcortical projecting (ET) cell
    "L5NP": "L5np", # Layer 5 near projecting (NP) cell
    "L6short-a": "L6it",  # Layer 6 cortico-cortical (IT) cell
    "L6short-b": "L6it",
    "L6tall-a": "L6ct", # Layer 6 cortico-thalamic (CT) cell
    "L6tall-b": "L6ct",
    "L6tall-c": "L6ct",
}

soma_df['cell_type'] = soma_df['cell_type'].replace(ct_simplify)
```

**Tasks**:
1) Run the simplification code above to replace cell type labels.
2) Based on this table, what fraction of all neurons are inhibitory?
3) What is the most numerous excitatory cell type?

## Reorienting the data

We are going to use a package called `standard_transform` to make it easier to put the data into a useful coordinate system.
The current `pt_position` values are in voxels and can be directly pasted into Neuroglancer.
However, usually for analysis we want spatial dimensions like nanometers or microns.
Moreover, it's useful if convenient directions like pia-to-white-matter axis correspond to the y-coordinate.

**Tasks**

Plot the x-y distribution of excitatory neuron cell body locations.
Note density as a function of depth, but also slight rotation of dataset.
_Hint_: The y-coordinates *increase* as you go down from pia.

Next, use the standard_transform function `minnie_ds.transform_vx.apply_dataframe( COLUMN_NAME, DATAFRAME )` that maps voxel coordinates in the column "COLUMN_NAME" of a dataframe to microns.
The transform will rotate the data by 5 degrees and align y=0 to be near the pial surface.
Plot the new x-y distribution of excitatory neuron cell body locations and see how that differs from the untransformed.

In [ ]:
from standard_transform.datasets import minnie_ds

...

### Soma depths and cell types

You can use the optional argument `projection="y"` for the function `minnie_ds.transform_vx.apply_dataframe` to get just the soma depth value and assign it to a column.

**Question**
What excitatory cell types overlap one another most in depth?

In [ ]:
...

## Different visual areas

The MICrONs dataset was taken at the edge of primary visual cortex and higher visual areas RL and AL.
We will now look at how cells might differ between primary and higher-order regions.
A function that returns True if a location is beyond a linear approximation of the HVAs and False otherwise.

**Task**

Using the code below, determine if each cell body is a in the HVA or not.

In [ ]:

# These values come from a linear approximation of primary visual cortex/HVA
# boundaries in *voxels* before transformation. Make sure to convert units if you want. 
xz0 = [237415, 26308]
xz1 = [286783, 8960]

x0 = xz0[0]
x1 = xz1[0]
z0 = xz0[1]
z1 = xz1[1]

def soma_in_hva(pt):
    ptz = pt[:,2]
    ptx = pt[:,0]
    x_thresh = x1 + (ptz-z1) * (x0-x1) / (z0-z1)
    return ptx > x_thresh


**Tasks**
1) For some location in primary visual cortex and some location in an HVA, compute the excitatory cell density measuring number of cells per unit volume as a function of depth. Depth bins or windows around 30-50 microns work well. Plot a comparison of the cell density profile of the HVA to VISp.

2) Compare the relative fraction of excitatory neurons with different cell types. What cell types are much more common in the HVA and which less common?

# Extra Credit!

As a bonus to help think about the data if you would like, I've pre-computed density profiles every 10 microns through a region along the center of the dataset.
This exercise is purely optional.
Using whatever data analysis technique you can think of, what areas of the data have similar density profiles?
How do these match with the VISp/HVA boundary?

The code used to compute density is shown below and the file is in this directory as `density_profile.feather`.
You can load this file with `profile_df = pd.read_feather('density_profile.feather')`.



```python
xvals = np.arange(450, 1301, 10)
zvals = np.arange(650, 1051, 10)

depths = np.linspace(0,800,100)

profiles = []
xloc = []
zloc = []
for xx in tqdm(xvals):
    for zz in zvals:
        profiles.append(
            cell_density(
                (xx, zz),
                depths,
                soma_df.query('classification_system == "excitatory_neuron"'),
                bin_height=40,
                width=50,
            )
        )
        xloc.append(xx)
        zloc.append(zz)

profile_df = pd.DataFrame(
    {
        "xloc": xloc,
        "zloc": zloc,
        "profile": profiles
    }
)

data_df = {"xloc": xloc, "zloc": zloc}
profiles = np.array(profiles)
profile_cols = []
for ii in range(profiles.shape[1]):
    data_df[f"dep_dens_{ii}"] = profiles[:,ii]
    profile_cols.append(f"dep_dens_{ii}")

profile_df = pd.DataFrame(data_df)
```

In [ ]:
# Load the file described above
profile_df = pd.read_feather('density_profile.feather')